In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.plotting.backend = 'plotly'
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from inputs.constants import DATA_PATH

In [ ]:
df = pd.read_csv(DATA_PATH / 'aus_vax_data.csv', index_col=424)
df.index = pd.to_datetime(df.index, infer_datetime_format=True)

In [ ]:
age_16_3dose = [
    col for col in df.columns if 
    'National' in col and 
    any([s in col for s in ['who have received 3 doses', 'with 3 or more doses', 'with more than two doses', 'with more than 2 doses']]) and 
    any([s in col for s in ['16', '18']]) and
    not any([s in col for s in ['increase', 'Percentage', 'Indigenous']])
]
age_16_4dose = [
    col for col in df.columns if
    'National' in col and
    any([s in col for s in ['Winter Boosters number', 'who have received 4 doses', 'Fourth dose number']]) and
    not any([s in col for s in ['30', '65', 'increase']])
]
age_12_15_2dose = [
    col for col in df.columns if
    '12-15' in col and
    any([s in col for s in ['National', 'Age group']]) and
    any([s in col for s in ['2 doses', 'fully vaccinated']]) and
    not any([s in col for s in ['Indigenous', 'Population', '- F -', '- M -']])
]
age_5_11_2dose = [
    col for col in df.columns if
    'National' in col and
    '5-11' in col and
    any([s in col for s in ['2 doses', 'fully vaccinated']])
]
age_16_booster = [
    col for col in df.columns if
    'National' in col and
    any([s in col for s in ['who have received 3 doses', 'with 3 or more doses', 'with more than two doses', 'with more than 2 doses', '3 or more doses', 'Winter Boosters', 'who have received 4 doses', 'Fourth dose number']]) and
    not any([s in col for s in ['Indigenous', 'increase', '30', '65', 'Percentage']])
]

In [ ]:
col_masks = {
    'Ages 16+, at least three doses': age_16_3dose,
    'Ages 16+, at least four doses': age_16_4dose,
    'Ages 12 to 15, at least two doses': age_12_15_2dose,
    'Ages 5 to 11, at least two doses': age_5_11_2dose,
}
fig = make_subplots(rows=2, cols=2, subplot_titles =list(col_masks.keys()))
for m, mask in enumerate(col_masks):
    row = m % 2 + 1
    col = int(np.floor(m / 2)) + 1
    fig.add_traces(px.line(df[col_masks[mask]]).data, rows=row, cols=col)
fig.update_layout(height=600, showlegend=False)

In [ ]:
booster_df = df[age_16_booster].drop([datetime(2022, 7, 8)])  # If we don't do this, we get a downward spike, which appears to be artefact
booster_df['total'] = booster_df.sum(axis=1)
booster_df['smoothed_total'] = booster_df.loc[:, 'total'].rolling(14).mean()
booster_df.plot()

In [ ]:
px.line(df[[c for c in df.columns if 'Number of people fully vaccinated' in c]].dropna(), title='2-dose coverage')

In [ ]:
data_cols_5_11 = [c for c in df.columns if '5-11' in c]
for c in data_cols_5_11:
    print(c)

In [ ]:
for name in list(df.columns.values):
    if 'booster' in name:
        print(name)